In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
import numpy as np
import string
import random
from SoundsLike.SoundsLike import Search
import Levenshtein

In [31]:
df = pd.read_csv("../../data/conll03/new_conll_dev_preprocessed_without_punctuation.csv")
#df = pd.read_csv("../../data/conll03/new_conll_train_preprocessed_without_punctuation_with_broken_entities.csv")
#df = pd.read_csv("../../data/conll03/train.txt", sep=" ")
#df = pd.read_csv("data/dev.txt", sep=" ")
#df = pd.read_csv("data/test.txt", sep=" ")

In [32]:
df.head(100)

,Unnamed: 0,sentence_no,Word,Tag
0,0,0,CRICKET,O
1,1,0,LEICESTERSHIRE,ORG
2,2,0,TAKE,O
3,3,0,OVER,O
4,4,0,AT,O
5,5,0,TOP,O
6,6,0,AFTER,O
7,7,0,INNINGS,O
8,8,0,VICTORY,O
9,9,1,LONDON,LOC


In [33]:
df = df[df['Tag'].notna()]

In [34]:
df.head()

,Unnamed: 0,sentence_no,Word,Tag
0,0,0,CRICKET,O
1,1,0,LEICESTERSHIRE,ORG
2,2,0,TAKE,O
3,3,0,OVER,O
4,4,0,AT,O


In [35]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [36]:
df['Tag'].unique()

array(['O', 'ORG', 'LOC', 'PER'], dtype=object)

In [37]:
len(df)

44268

In [38]:
df.dropna(axis=0, inplace=True)

In [39]:
df[df['Word'].isna()]

,sentence_no,Word,Tag


In [40]:
df.head(100)

,sentence_no,Word,Tag
0,0,CRICKET,O
1,0,LEICESTERSHIRE,ORG
2,0,TAKE,O
3,0,OVER,O
4,0,AT,O
5,0,TOP,O
6,0,AFTER,O
7,0,INNINGS,O
8,0,VICTORY,O
9,1,LONDON,LOC


In [14]:
dataset = []
for sentence_no, tag, word in zip(df['sentence_no'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if tag != "O":
        error = []
        try:
            x = Search.perfectHomophones(word)
            for d in x:
                err = Levenshtein.distance(word, d)
                error.append(err)
                word = x[error.index(max(error))]
        except ValueError:
            pass 
        dataset.append((sentence_no, tag, word))
    else:
        dataset.append((sentence_no, tag, word))
df = pd.DataFrame(dataset, columns=['Sentence #', 'Tag', 'Word'])

In [18]:
df.head(100)

,Sentence #,Tag,Word
0,0,ORG,Eu
1,0,O,rejects
2,0,O,German
3,0,O,call
4,0,O,to
5,0,O,boycott
6,0,O,British
7,0,O,lamb
8,1,PER,Pieter
9,1,PER,Blackburn


In [15]:
old_df = df

In [16]:
entities = df[df['Tag'] != "O"]['Sentence #'].values.tolist()

In [17]:
per_entities = df[df['Tag'] == "PER"]['Sentence #'].values.tolist()

In [18]:
per_entities

[1,
 1,
 3,
 3,
 4,
 5,
 5,
 5,
 5,
 6,
 7,
 8,
 8,
 9,
 9,
 11,
 11,
 11,
 11,
 11,
 11,
 13,
 13,
 13,
 13,
 17,
 17,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 29,
 30,
 30,
 31,
 31,
 31,
 31,
 32,
 32,
 33,
 33,
 33,
 34,
 34,
 35,
 37,
 37,
 37,
 38,
 38,
 38,
 42,
 42,
 42,
 42,
 42,
 42,
 44,
 44,
 48,
 48,
 49,
 49,
 49,
 49,
 49,
 50,
 50,
 50,
 51,
 51,
 55,
 58,
 58,
 59,
 59,
 59,
 59,
 60,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,


In [24]:
per_randomlist = random.sample(per_entities, int(len(per_entities)/2))
per_randomlist.sort()

In [25]:
len(per_randomlist)

11124

In [19]:
for ind, sentence_no, tag, word in zip(df.index.tolist(), df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if sentence_no in per_entities and tag == "PER":
        row1, row3 = df.iloc[ind-1].copy(), df.iloc[ind+1].copy()
        df.iloc[ind-1], df.iloc[ind+1] = row3, row1

In [20]:
df.head(100)

,Sentence #,Tag,Word
0,0,O,CRICKET
1,0,ORG,LEICESTERSHIRE
2,0,O,TAKE
3,0,O,OVER
4,0,O,AT
5,0,O,TOP
6,0,O,AFTER
7,0,O,INNINGS
8,0,O,VICTORY
9,1,LOC,London


In [21]:
loc_entities = df[df['Tag'] == "LOC"]['Sentence #'].values.tolist()

In [22]:
loc_entities

[1,
 3,
 3,
 3,
 5,
 6,
 8,
 8,
 9,
 16,
 16,
 19,
 20,
 20,
 21,
 21,
 21,
 22,
 22,
 23,
 23,
 25,
 25,
 25,
 26,
 27,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 29,
 30,
 30,
 32,
 32,
 40,
 40,
 40,
 40,
 40,
 41,
 41,
 41,
 42,
 42,
 42,
 43,
 47,
 47,
 48,
 49,
 49,
 50,
 50,
 59,
 59,
 60,
 60,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 64,
 66,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 69,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 70,
 71,
 71,
 72,
 72,
 76,
 79,
 79,
 82,
 113,
 119,
 119,
 123,
 123,
 123,
 123,
 123,
 123,
 123,
 123,
 123,
 123,
 123,
 124,
 124,
 148,
 149,
 149,
 149,
 152,
 15

In [ ]:
loc_randomlist = random.sample(loc_entities, int(len(loc_entities)/2))
loc_randomlist.sort()

In [23]:
for ind, sentence_no, tag, word in zip(df.index.tolist(), df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if sentence_no in loc_entities and tag == "LOC":
        row1, row3 = df.iloc[ind-1].copy(), df.iloc[ind+1].copy()
        df.iloc[ind-1], df.iloc[ind+1] = row3, row1

In [24]:
org_entities = df[df['Tag'] == "ORG"]['Sentence #'].values.tolist()

In [ ]:
org_randomlist = random.sample(org_entities, int(len(org_entities)/2))
org_randomlist.sort()

In [25]:
for ind, sentence_no, tag, word in zip(df.index.tolist(), df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if sentence_no in org_entities and tag == "ORG":
        row1, row3 = df.iloc[ind-1].copy(), df.iloc[ind+1].copy()
        df.iloc[ind-1], df.iloc[ind+1] = row3, row1

In [26]:
df.head(100)

,Sentence #,Tag,Word
0,0,O,TAKE
1,0,ORG,LEICESTERSHIRE
2,0,O,CRICKET
3,0,O,OVER
4,0,O,AT
5,0,O,TOP
6,0,O,AFTER
7,0,O,INNINGS
8,1,O,19960830
9,1,LOC,London


In [27]:
new_df = df.copy()

In [28]:
new_df.head()

,Sentence #,Tag,Word
0,0,O,TAKE
1,0,ORG,LEICESTERSHIRE
2,0,O,CRICKET
3,0,O,OVER
4,0,O,AT


In [29]:
new_df.drop('Sentence #', axis=1, inplace=True)

In [30]:
new_df.head()

,Tag,Word
0,O,TAKE
1,ORG,LEICESTERSHIRE
2,O,CRICKET
3,O,OVER
4,O,AT


In [41]:
new_df['Sentence #'] = df['sentence_no']

In [42]:
new_df.tail()

,Tag,Word,Sentence #
44260,O,on,1873.0
44261,ORG,Newsroom,1873.0
44262,O,8802506363,1873.0
44263,O,Thursday,1873.0
44264,ORG,Dhaka,1873.0


In [43]:
new_df.to_csv("../../data/conll03/new_conll_dev_preprocessed_without_punctuation_with_broken_entities_broken_context1.csv")